In [1]:
### step 1 build the folders with the picked document to be analyzed in them
            ### (manually) choose the file type to analyse (.txt or .html)
            ### create output repository
### step 2 overview: check classify document as normal, large,over oversized
            ### < 14,000 tk    normal
            ### 14,000 < ... < 150,000 tk large
            ### > 150,000 tk oversized
### step 3 normal sized summary production
            ### construct message
            ### gpt request
### step 4 large sized summary production
            ### segmentation
            ### construct message (inside write summary)
            ### write summary
### step 5 JSON profile production
            ### construct prompts with examples (few-shot)
            ### write JSON profile


In [2]:
### 
### all files to run:
### Austria, Italy, Luxembourg, Belgium
### only large files to run:
### Spain, Poland
### finished:
### France, Portugal, Netherlands

In [4]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import pathlib
from pathlib import Path
import json
import os
import tiktoken
from bs4 import BeautifulSoup

In [5]:
import openai
from openai import OpenAI

In [6]:
import prompt_text as ppt
import summary_scripts as ss

In [7]:
# client = OpenAI(api_key='<your api_key>')

In [40]:
### hyper parameters
COMPLETIONS_MODEL = "gpt-3.5-turbo-1106"
MAX_TOKEN_SUMMARY = 1000
CHUNK_SIZE = 5000
NORMAL_TOKEN_DOC = 14000
MAX_TOKEN_DOC = 150000
TEMPERATURE = 1.0

### Austria, Italy, Belgium ###

In [9]:
### read in es.txt file
def read_lines(file):
    with open(file) as f:
        lines = [l.strip() for l in f.readlines() if '---  page ' not in l and not l.strip() == '']
    return lines

### input path to the file
def read_txt(file):
    with open(file) as f:
        lines = [l.strip() for l in f.readlines() if '---  page ' not in l and not l.strip() == '']
        txt = ' '.join(lines)
    return txt

### input path to the file
def read_html(file):
    with open(file, 'r', encoding="utf-8") as f:
        t = f.read()
    soup = BeautifulSoup(t, 'html.parser')
    txt = soup.get_text()
    return txt

In [33]:
### create repository and check the number of big files/oversize files
def create_output_repository(src_dir, trg_dir,file_suffix):
    path_src_list  = [] # collection of paths to all case input folders
    dir_dst_list = [] #  collection of paths to all case output folders
    c = 0
    sum = 0
    num_big_files = 0
    num_too_big_files = 0
    for root, dirs, files in os.walk(src_dir):
        for f in files:
            if f.endswith(file_suffix):
                filepath = os.path.join(root, f)
                path = pathlib.PurePath(filepath)
                parent = path.parent.name
                ### only deal with cases after 2019
                year = parent[0:4]
                if not year.isdigit() or int(parent[0:4]) < 2019:
                    continue
                c += 1
                grandpa = path.parent.parent.name
                dir_grand  = trg_dir+grandpa
                dir_parent = trg_dir+grandpa+"/"+parent
                if file_suffix.endswith(".txt"):
                    txt = read_txt(filepath)
                elif file_suffix.endswith(".html"):
                    txt = read_html(filepath)
                else:
                    print("ERROR: file type not implemented")
                a = ss.count_tokens(txt)
                if a  > 14000 and a < 150000:
                    num_big_files += 1
                    print("big file with tokens:" + str(a))
                elif a >= 150000:
                    num_too_big_files += 1
                    print("oversized big file with tokens:" + str(a))
                else:
                    # print("normal sized file with tokens:" + str(a))
                    sum += a
                if not os.path.exists(dir_grand):
                    os.makedirs(dir_grand)
                if not os.path.exists(dir_parent):
                    os.makedirs(dir_parent)
                save_doc = dir_parent + "/" + f
                if not os.path.exists(save_doc):
                    with open(save_doc, "w") as f:
                        f.write(txt)
    print("----- total number of files after 2019: " + str(c))
    print("----- number of large files: " + str(num_big_files))
    print("----- number of oversized files: " + str(num_too_big_files))
    print("----- average tokens per document (excluding large and oversized files): " + str(sum/c))

In [60]:
### simple segmentation function
def partition_document(txt, CHUNK_SIZE = 5000):

    token_count = ss.count_tokens(txt)

    num_segments = int(np.round(token_count / CHUNK_SIZE))
    print(num_segments)

    char_count = len(txt)
    split_length = int(np.round(char_count/num_segments))
    # print(split_length)

    segments = []
    c = 0
    head = 0
    tail = split_length * 1
    temp = tail
    find_split = False
    while(c < num_segments):
        find_split = False
        if c == num_segments - 1:
            segments.append(txt[head:])
            c += 1
        else:
            while(not find_split):
                if txt[temp:temp+2] == '. ':
                    tail =  temp + 1
                    find_split = True
                    segments.append(txt[head:tail])
                    head = tail + 1
                    temp = head + split_length * 1
                    tail = temp
                    c += 1
                    break
                else:
                    temp += 1
    return segments

In [11]:
def get_completion(doc,prompt, msg_sys, country = "spain", modelname = "gpt-3.5-turbo-1106", temp = 1.0,max_tokens = 1000):
    mssges = [{"role" : "system", "content": msg_sys}]
    
    # if show_prompt:
    #     print(prompt)
    mssges.append({"role" : "user", "content":prompt + doc})
    completion = client.chat.completions.create(
        model=modelname,
        messages= mssges,
        max_tokens=max_tokens,
        temperature = temp)

    #print tokens used in prompt and completion
    print("Tokens in prompt: ", completion.usage.prompt_tokens)
    print("Tokens in completion: ", completion.usage.completion_tokens)

    return completion

In [12]:
def get_completion_long(mssges, modelname = "gpt-3.5-turbo-1106", temp = 1.0,max_tokens = 1000):
    '''
    a simplified version of get_completion
    mssges should be prepared as argument to completion request
    '''
    completion = client.chat.completions.create(
        model=modelname,
        messages= mssges,
        max_tokens=max_tokens,
        temperature = temp)

    #print tokens used in prompt and completion
    print("Tokens in prompt: ", completion.usage.prompt_tokens)
    print("Tokens in completion: ", completion.usage.completion_tokens)

    return completion

In [58]:
def write_summary(doc,savedir,country,save_result = True,token_limit = 14000, model_name = "gpt-3.5-turbo-1106", t = 1.0):
    '''
    Summarizes the text in the file at filepath and saves the summary and completion object to savedir
    filepath: path to text file
    savedir: directory where to save the summary and completion object
    country: country for which to summarize
    token_limit: maximum number of tokens for the text
    modelname: name of the model to use

    Returns summary and completion object
    '''

    # if filepath.endswith(".html"):
    #     text = html2str(filepath)
    #     savepath_original = os.path.join(savedir, "original.html")

    #     #save the original file in savedir if it does not exist yet
    #     if not os.path.exists(savepath_original):
    #         with open(filepath, 'r', encoding="utf-8") as f:
    #             original_text = f.read()

    #         with open(savepath_original, "w") as f:
    #             f.write(original_text)

    # else: raise NotImplementedError(f"Filetype {filepath} not implemented")
    
    #count text tokens
    tokens_text = ss.count_tokens(doc, modelname=model_name)

    if tokens_text >= token_limit:
        print("Tokens in text: ", tokens_text)
        print(f"Text too long: Longer than {token_limit} tokens. Returning empty string and empty dictionary.")
        return '' , {}
    
    completion = get_completion(doc,ppt.prpt_bullets_ada, ppt.role_system_bullets, country,modelname = model_name, temp = t) ### to-do: write get completion
    summary = completion.choices[0].message.content

    completion_dict = ss.convert_chat_completion_to_dict(completion)

    if save_result:

        savepath_summary = os.path.join(savedir, "summary_t" + str(t) + ".txt")
        savepath_completion = os.path.join(savedir, "completion.json")

        completion_json = json.dumps(completion_dict)

        #save the summary and the completion object in the savedir

        with open(savepath_completion, "w") as f:  
            f.write(completion_json)

        with open(savepath_summary, "w") as f:
            f.write(summary)
    
    return summary, completion_dict

In [62]:
def write_summary_long(segments,savedir,country,save_result = True,token_limit = 14000, model_name = "gpt-3.5-turbo-1106", t = 1.0):
    '''
    Summarizes a sequence of segments from a large document progressively
    

    segments: [text1,text2,...], a list of consecutive segments from a long document
    savedir: directory where to save the summary and completion object
    country: country for which to summarize
    token_limit: maximum number of tokens for the text (not used in this case)
    modelname: name of the model to use

    Returns a list of summaries. The last item is the final and complete version. 
    '''
    ### prepare message for the first segment summary
    msgs_start = [{"role" : "system", "content": ppt.role_system_bullets}]
    content_start = ppt.prpt_bullets_ada + segments[0]
    msgs_start.append({"role" : "system", "content": content_start})

    print("--- summary chain starts ------")
    comp_start = get_completion_long(msgs_start)
    summary_chain = [] # the list of all segment summaries
    summ_temp = comp_start.choices[0].message.content
    summary_chain.append(summ_temp)
    n_ch = len(segments)
    ### progressive summary revision starts
    # summary_(i+1) is based on segment_(i+1),summary_(i) 
    for i in range(n_ch-1):
        idx = i + 1
        print("--- chain at " + str(idx) + "/"+ str(n_ch))
        msgs_chain = [{"role" : "system", "content": ppt.role_system_bullets_chain}]
        content_chain = ppt.old_summ_chain + summ_temp + ppt.prpt_bullets_ada_chain + segments[idx] + ppt.prpt_chain
        msgs_chain.append({"role" : "user", "content": content_chain})
        comp_chain = get_completion_long(msgs_chain)
        summ_temp = comp_chain.choices[0].message.content
        completion_dict = ss.convert_chat_completion_to_dict(comp_chain)
        # print(summ_temp)
        summary_chain.append(summ_temp)

        if save_result:

            savepath_summary = os.path.join(savedir, "summary_t" + str(t) + "_seg" + str(idx) + ".txt")
            if idx == n_ch - 1:
                savepath_summary = os.path.join(savedir, "summary_t" + str(t) + ".txt")
            savepath_completion = os.path.join(savedir, "completion_seg"+ str(idx)+ ".json")

            completion_json = json.dumps(completion_dict)

            #save the summary and the completion object in the savedir

            with open(savepath_completion, "w") as f:  
                f.write(completion_json)

            with open(savepath_summary, "w") as f:
                f.write(summ_temp)
    
    print("Total number of summaries written: " + str(len(summary_chain)))
    return summary_chain

In [15]:
def construct_message_summary_to_json(summary,msg_sys,example_inputs = [], example_outputs = []):
    message = []
    message.append({"role": "system", "content": msg_sys})

    for i in range(len(example_inputs)):
        message.append({"role": "user", "content": example_inputs[i]})
        message.append({"role": "assistant", "content": example_outputs[i]})

    message.append({"role": "user", "content": summary})

    return message

In [16]:
def summary_to_json(mssges, savedir, modelname = "gpt-3.5-turbo-1106", save_json = True, temp = 1.0, max_tokens = 500):
    
    completion = client.chat.completions.create(
            model=modelname,
            response_format={ "type": "json_object" },
            messages= mssges,
            max_tokens= max_tokens,
            temperature=temp)
    
            #print tokens used in prompt and completion
    print("Tokens in prompt: ", completion.usage.prompt_tokens)
    print("Tokens in completion: ", completion.usage.completion_tokens)

    try:
        response_dict = json.loads(completion.choices[0].message.content.replace('\n',''))
    except Exception as e:
        print(e)
        print("API response could not be converted to dictionary. Returning the content of the completion message and the completion object as dict")
        return completion.choices[0].message.content, ss.convert_chat_completion_to_dict(completion)
    
    if save_json:
        savepath_json = savedir + '/tj' + str(temp) +".json"
        with open(savepath_json, "w") as f:
            json.dump(response_dict, f)

    return response_dict, ss.convert_chat_completion_to_dict(completion)

In [23]:
### Austria
SRC_DIR_aus = './DPA-decisions/DPA-decisions/AUSTRIA/' #full path to source directory
TRG_DIR_aus = './austria_qr_results/Datenschutzbehorde/'#full path to target directory
SUM_DIR_aus = TRG_DIR_aus
suffix_aus = '.pdf.de.txt'


In [24]:
### Italy
SRC_DIR_ita = './DPA-decisions/DPA-decisions/ITALY/' #full path to source directory
TRG_DIR_ita = './italy_qr_results/GPDP/'#full path to target directory
SUM_DIR_ita = TRG_DIR_ita
suffix_ita = 'document.html'

In [26]:
### Luxembourg only gives jsons 
# SRC_DIR_lux = './DPA-decisions/DPA-decisions/LUXEMBORG/' #full path to source directory
# TRG_DIR_lux= './luxembourg_qr_results/VDAI/'#full path to target directory
# SUM_DIR_lux = TRG_DIR_lux

In [25]:
### Belgium
SRC_DIR_bel = './DPA-decisions/DPA-decisions/BELGIUM/' #full path to source directory
TRG_DIR_bel = './belgium_qr_results/DPA/'#full path to target directory
SUM_DIR_bel = TRG_DIR_bel
suffix_bel = 'pdf.nl.txt'

In [22]:
### Hungary TO-DO (might contain too many large documents)
SRC_DIR_hun = './DPA-decisions/DPA-decisions/HUNGARY/' #full path to source directory
TRG_DIR_hun = './hungary_qr_results/DPA/'#full path to target directory
SUM_DIR_hun = TRG_DIR_hun

In [35]:
create_output_repository(SRC_DIR_aus,TRG_DIR_aus,suffix_aus)

big file with tokens:14235
big file with tokens:19377
big file with tokens:16051
----- total number of files after 2019: 67
----- number of large files: 3
----- number of oversized files: 0
----- average tokens per document (excluding large and oversized files): 5370.089552238806


In [34]:
create_output_repository(SRC_DIR_ita,TRG_DIR_ita,suffix_ita)

big file with tokens:35315
big file with tokens:16231
big file with tokens:14426
big file with tokens:18559
big file with tokens:33790
big file with tokens:15631
big file with tokens:22517
big file with tokens:17118
big file with tokens:14157
big file with tokens:30771
big file with tokens:26294
big file with tokens:20522
big file with tokens:14372
big file with tokens:31912
big file with tokens:35503
big file with tokens:15602
big file with tokens:20755
big file with tokens:14073
big file with tokens:37131
big file with tokens:16636
big file with tokens:16130
big file with tokens:40768
big file with tokens:15571
big file with tokens:25078
big file with tokens:31300
big file with tokens:14015
big file with tokens:51782
----- total number of files after 2019: 341
----- number of large files: 27
----- number of oversized files: 0
----- average tokens per document (excluding large and oversized files): 6256.882697947214


In [36]:
create_output_repository(SRC_DIR_bel,TRG_DIR_bel,suffix_bel)

big file with tokens:22134
big file with tokens:101485
big file with tokens:31218
big file with tokens:22172
big file with tokens:14041
big file with tokens:26019
big file with tokens:15767
big file with tokens:25240
big file with tokens:23908
big file with tokens:14981
big file with tokens:25248
big file with tokens:41361
big file with tokens:18200
big file with tokens:33828
big file with tokens:18315
big file with tokens:16483
big file with tokens:26806
big file with tokens:22832
big file with tokens:15342
big file with tokens:23313
big file with tokens:24442
big file with tokens:25945
big file with tokens:21137
big file with tokens:16548
big file with tokens:29865
big file with tokens:22932
big file with tokens:16062
big file with tokens:17779
big file with tokens:14581
big file with tokens:34298
big file with tokens:14567
big file with tokens:18804
big file with tokens:30986
big file with tokens:16193
big file with tokens:14696
big file with tokens:29409
big file with tokens:20282


In [63]:
def produce_summary_pipeline(sum_dir,file_suffix):
    ### normal sized files.
    summ_count = 0 ### number of total summaries produced
    large_summ_count = 0
    for root, dirs, files in os.walk(sum_dir):
        for f in files:
            if f.endswith(file_suffix):
                filepath = os.path.join(root, f)
                path = pathlib.PurePath(filepath)
                parent = path.parent.name
                grandpa = path.parent.parent.name
                dir_grand  = sum_dir+grandpa
                dir_parent = sum_dir+grandpa+"/"+parent

                ## read in
                if file_suffix.endswith(".txt"):
                    txt = read_txt(filepath)
                elif file_suffix.endswith(".html"):
                    txt = read_html(filepath)
                else:
                    print("ERROR: file type not implemented")
                    continue
                
                ### write summary
                token = ss.count_tokens(txt)
                if token > NORMAL_TOKEN_DOC and token < MAX_TOKEN_DOC:
                    summ_count += 1
                    large_summ_count += 1
                    print("--- big file with tokens:" + str(token) + " ------")
                    #### large document pipeline
                    segments = partition_document(txt)
                    ### write summary
                    print("--- directory: " + str(dir_parent)+ " ------")
                    summary_chain = write_summary_long(segments,dir_parent,country="portugal", save_result = True, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                    # summary_aggr.append(summary_chain)
                elif token < NORMAL_TOKEN_DOC:
                    summ_count += 1
                    ### write summary
                    summary, completion_dict = write_summary(txt,dir_parent,country=" ", save_result = True,token_limit = NORMAL_TOKEN_DOC, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                else:
                    print("Document too large with size " + str(token) + " exceeding the limit" + str(MAX_TOKEN_DOC))
                
                
                if summ_count % 20 == 0:
                    print(" ----- SUMMARY FINISHED: "+  str(summ_count) + " -----")
                    
    print("----- END:::::::::: Total summaries finished -----" + str(summ_count))
    print("----- END:::::::::: Large doc summaries finished: -----" + str(large_summ_count))

In [50]:
examples_summary = [ppt.summary_ex1.replace('\n', ' '),ppt.summary_ex2.replace('\n', ' '),ppt.summary_ex3.replace('\n', ' ')]
examples_json = [ppt.target_out_1.replace('\n', ' '),ppt.target_out_2.replace('\n', ' '),ppt.target_out_3.replace('\n', ' ')]

In [56]:

"""
HERE PRODUCE THE JSON FROM THE SUMMARY
"""
def produce_json_pipeline(sum_dir,exp_summary, exp_json):
    c_summ = 0
    c_json = 0
    for root, dirs, files in os.walk(sum_dir):
        for f in files:
            if f == "summary_t1.0.txt":
                c_summ +=1
                filepath = os.path.join(root, f)
                summa = read_txt(filepath)
                path = pathlib.PurePath(filepath)
                parent = path.parent.name
                grandpa = path.parent.parent.name
                save_dir = sum_dir + grandpa+"/"+parent
                # print(save_dir)
                msg = construct_message_summary_to_json(summa, ppt.msg_sys_json, exp_summary, exp_json)
                res_dict, comp = summary_to_json(msg, save_dir)
                if not isinstance(res_dict, str):
                    c_json += 1
                # print(res_dict)
                # res_list.append(res_dict)
                # comp_list.append(comp)
                if c_json % 20 == 0:
                    print(" ----- JSON FINISHED: "+  str(c_json) + " -----")
                    # print(summary)
                    # print(completion_dict)
                    # with open(dst_path,'w') as f: ### write the 
                    #     pass
                # print(dir_grand)
                # print(dir_parent)
                # print(dst_path)
    print("----- summaries found in this folder: ------" + str(c_summ))
    print("----- json profiles produced: ------" + str(c_json))

In [64]:
produce_summary_pipeline(SUM_DIR_aus,suffix_aus)

Tokens in prompt:  7994
Tokens in completion:  580
Tokens in prompt:  4117
Tokens in completion:  388
Tokens in prompt:  3183
Tokens in completion:  303
Tokens in prompt:  4167
Tokens in completion:  396
Tokens in prompt:  7081
Tokens in completion:  507
Tokens in prompt:  3825
Tokens in completion:  331
Tokens in prompt:  8004
Tokens in completion:  486
Tokens in prompt:  7739
Tokens in completion:  309
Tokens in prompt:  9430
Tokens in completion:  345
Tokens in prompt:  2820
Tokens in completion:  437
Tokens in prompt:  8173
Tokens in completion:  331
Tokens in prompt:  7273
Tokens in completion:  390
Tokens in prompt:  4080
Tokens in completion:  409
Tokens in prompt:  6017
Tokens in completion:  282
Tokens in prompt:  3394
Tokens in completion:  387
Tokens in prompt:  2580
Tokens in completion:  334
Tokens in prompt:  6928
Tokens in completion:  342
Tokens in prompt:  10374
Tokens in completion:  384
Tokens in prompt:  6883
Tokens in completion:  435
Tokens in prompt:  2637
Tokens

In [65]:
produce_json_pipeline(SUM_DIR_aus, examples_summary,examples_json)

Tokens in prompt:  4223
Tokens in completion:  428
Tokens in prompt:  4031
Tokens in completion:  323
Tokens in prompt:  3946
Tokens in completion:  283
Tokens in prompt:  4039
Tokens in completion:  332
Tokens in prompt:  4150
Tokens in completion:  427
Tokens in prompt:  3965
Tokens in completion:  313
Tokens in prompt:  4129
Tokens in completion:  432
Tokens in prompt:  3952
Tokens in completion:  302
Tokens in prompt:  3988
Tokens in completion:  366
Tokens in prompt:  4079
Tokens in completion:  354
Tokens in prompt:  3974
Tokens in completion:  334
Tokens in prompt:  4033
Tokens in completion:  412
Tokens in prompt:  4052
Tokens in completion:  301
Tokens in prompt:  3926
Tokens in completion:  321
Tokens in prompt:  4030
Tokens in completion:  305
Tokens in prompt:  3977
Tokens in completion:  304
Tokens in prompt:  3985
Tokens in completion:  352
Tokens in prompt:  4018
Tokens in completion:  447
Tokens in prompt:  4072
Tokens in completion:  415
Tokens in prompt:  4065
Tokens 

In [66]:
produce_summary_pipeline(SUM_DIR_ita,suffix_ita)

Tokens in prompt:  10615
Tokens in completion:  445
Tokens in prompt:  12593
Tokens in completion:  255
Tokens in prompt:  7608
Tokens in completion:  445
Tokens in prompt:  2303
Tokens in completion:  482
Tokens in prompt:  2283
Tokens in completion:  505
Tokens in prompt:  2310
Tokens in completion:  432
Tokens in prompt:  3193
Tokens in completion:  507
Tokens in prompt:  2298
Tokens in completion:  487
Tokens in prompt:  2348
Tokens in completion:  442
Tokens in prompt:  2375
Tokens in completion:  519
Tokens in prompt:  7177
Tokens in completion:  508
Tokens in prompt:  2291
Tokens in completion:  494
Tokens in prompt:  4987
Tokens in completion:  432
Tokens in prompt:  8347
Tokens in completion:  457
Tokens in prompt:  2292
Tokens in completion:  327
Tokens in prompt:  2305
Tokens in completion:  413
Tokens in prompt:  5068
Tokens in completion:  519
Tokens in prompt:  2300
Tokens in completion:  429
Tokens in prompt:  2294
Tokens in completion:  521
Tokens in prompt:  4862
Token

In [68]:
produce_json_pipeline(SUM_DIR_ita, examples_summary,examples_json)

Tokens in prompt:  4079
Tokens in completion:  423
Tokens in prompt:  3898
Tokens in completion:  284
Tokens in prompt:  4088
Tokens in completion:  416
Tokens in prompt:  4125
Tokens in completion:  336
Tokens in prompt:  4148
Tokens in completion:  342
Tokens in prompt:  4075
Tokens in completion:  353
Tokens in prompt:  4150
Tokens in completion:  416
Tokens in prompt:  4130
Tokens in completion:  410
Tokens in prompt:  4085
Tokens in completion:  298
Tokens in prompt:  4162
Tokens in completion:  410
Tokens in prompt:  4142
Tokens in completion:  500
Unterminated string starting at: line 1 column 2152 (char 2151)
API response could not be converted to dictionary. Returning the content of the completion message and the completion object as dict
Tokens in prompt:  4137
Tokens in completion:  382
Tokens in prompt:  4075
Tokens in completion:  418
Tokens in prompt:  4099
Tokens in completion:  500
Unterminated string starting at: line 1 column 2137 (char 2136)
API response could not be

In [69]:
produce_summary_pipeline(SUM_DIR_bel,suffix_bel)

Tokens in prompt:  2229
Tokens in completion:  377
Tokens in prompt:  3532
Tokens in completion:  396
Tokens in prompt:  3698
Tokens in completion:  432
Tokens in prompt:  3060
Tokens in completion:  447
Tokens in prompt:  4174
Tokens in completion:  484
Tokens in prompt:  3971
Tokens in completion:  412
Tokens in prompt:  3519
Tokens in completion:  428
Tokens in prompt:  6818
Tokens in completion:  524
Tokens in prompt:  3339
Tokens in completion:  434
Tokens in prompt:  3835
Tokens in completion:  528
Tokens in prompt:  3603
Tokens in completion:  403
Tokens in prompt:  3884
Tokens in completion:  387
Tokens in prompt:  5533
Tokens in completion:  421
Tokens in prompt:  2659
Tokens in completion:  445
Tokens in prompt:  5388
Tokens in completion:  440
Tokens in prompt:  3503
Tokens in completion:  469
Tokens in prompt:  3474
Tokens in completion:  390
Tokens in prompt:  3700
Tokens in completion:  547
Tokens in prompt:  3559
Tokens in completion:  387
Tokens in prompt:  3288
Tokens 

In [70]:
produce_json_pipeline(SUM_DIR_bel, examples_summary,examples_json)

Tokens in prompt:  4020
Tokens in completion:  413
Tokens in prompt:  4039
Tokens in completion:  360
Tokens in prompt:  4075
Tokens in completion:  364
Tokens in prompt:  4090
Tokens in completion:  420
Tokens in prompt:  4127
Tokens in completion:  408
Tokens in prompt:  4055
Tokens in completion:  393
Tokens in prompt:  4071
Tokens in completion:  438
Tokens in prompt:  4167
Tokens in completion:  440
Tokens in prompt:  4076
Tokens in completion:  353
Tokens in prompt:  4171
Tokens in completion:  417
Tokens in prompt:  4046
Tokens in completion:  348
Tokens in prompt:  4021
Tokens in completion:  359
Tokens in prompt:  4064
Tokens in completion:  433
Tokens in prompt:  4088
Tokens in completion:  429
Tokens in prompt:  4083
Tokens in completion:  381
Tokens in prompt:  4111
Tokens in completion:  406
Tokens in prompt:  4033
Tokens in completion:  410
Tokens in prompt:  4190
Tokens in completion:  423
Tokens in prompt:  4030
Tokens in completion:  433
Tokens in prompt:  4042
Tokens 

In [ ]:
### Austria, Italy, Belgium, (Hungary), (Ireland) ###

### Spain (large document) ###

In [73]:
### Spain
SRC_DIR_es = './DPA-decisions/DPA-decisions/SPAIN/' #full path to source directory
TRG_DIR_es = './spain_qr_results/'#full path to target directory
SUM_DIR_es = TRG_DIR_es
suffix_es = 'document.txt'

In [89]:
### large document spain
summ_count = 0
num_big_files = 0
for root, dirs, files in os.walk(SUM_DIR_es):
    for f in files:
        if f.endswith(suffix_es):
            filepath = os.path.join(root, f)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR_es+grandpa
            dir_parent = TRG_DIR_es+grandpa+"/"+parent
            txt = read_txt(filepath)
            token = ss.count_tokens(txt)
            if token > NORMAL_TOKEN_DOC and token < MAX_TOKEN_DOC:
                print(dir_parent)
                num_big_files += 1
                summ_count += 1
                print("----- big file with tokens:" + str(token) + " ------")
                #### large document pipeline
                segments = partition_document(txt)
                ### write summary
                print("--- directory: " + str(dir_parent)+ " ------")
                summary_chain = write_summary_long(segments,dir_parent,country="spain", save_result = True, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                # summary_aggr.append(summary_chain)
                continue
            elif token <= NORMAL_TOKEN_DOC:
                summ_count += 1
                ### write summary
                # summary, completion_dict = write_summary(txt,dir_parent,country="portugal", save_result = True,token_limit = MAX_TOKEN_DOC, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                # ### produce examples for JSON
                # ### write JSON
                # # if summ_count % 20 == 0:
                # print(" ----- SUMMARY FINISHED: "+  str(summ_count) + " -----")
            elif token >= MAX_TOKEN_DOC:
                print("------------------------------------too huge file with tokens: " + str(token))
                summ_count += 1
            
print("----- number of large document summaries: -----" + str(num_big_files))
print("----- total number of documents-----" + str(summ_count))

./spain_qr_results/RESOLUCION_DE_TERMINACION_DEL_PROCEDIMIENTO_POR_PAGO/2022-07-06--PS_00611_2021
----- big file with tokens:15042 ------
3
--- directory: ./spain_qr_results/RESOLUCION_DE_TERMINACION_DEL_PROCEDIMIENTO_POR_PAGO/2022-07-06--PS_00611_2021 ------
--- summary chain starts ------
Tokens in prompt:  5654
Tokens in completion:  475
--- chain at 1/3
Tokens in prompt:  6070
Tokens in completion:  552
--- chain at 2/3
Tokens in prompt:  5900
Tokens in completion:  540
Total number of summaries written: 3
./spain_qr_results/RESOLUCION_DE_TERMINACION_DEL_PROCEDIMIENTO_POR_PAGO/2022-04-29--PS_00323_2021
----- big file with tokens:19467 ------
4
--- directory: ./spain_qr_results/RESOLUCION_DE_TERMINACION_DEL_PROCEDIMIENTO_POR_PAGO/2022-04-29--PS_00323_2021 ------
--- summary chain starts ------
Tokens in prompt:  5508
Tokens in completion:  527
--- chain at 1/4
Tokens in prompt:  5795
Tokens in completion:  535
--- chain at 2/4
Tokens in prompt:  5881
Tokens in completion:  542
--- c

In [97]:
### large document spain
large_summ_count = 0
c_json = 0
for root, dirs, files in os.walk(SUM_DIR_es):
    for f in files:
        if f.endswith(suffix_es):
            filepath = os.path.join(root, f)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR_es+grandpa
            dir_parent = TRG_DIR_es+grandpa+"/"+parent
            txt = read_txt(filepath)
            token = ss.count_tokens(txt)
            if token > NORMAL_TOKEN_DOC and token < MAX_TOKEN_DOC:
                # print(save_dir)
                summary_path = dir_parent + "/summary_t1.0.txt"
                summa = read_txt(summary_path)
                large_summ_count += 1
                msg = construct_message_summary_to_json(summa, ppt.msg_sys_json, examples_summary, examples_json)
                res_dict, comp = summary_to_json(msg, dir_parent)
                if not isinstance(res_dict, str):
                    c_json += 1
                # print(res_dict)
                # res_list.append(res_dict)
                # comp_list.append(comp)
                if c_json % 20 == 0:
                    print(" ----- JSON FINISHED: "+  str(c_json) + " -----")
                    # print(summary)
                    # print(completion_dict)
                    # with open(dst_path,'w') as f: ### write the 
                    #     pass
                # print(dir_grand)
                # print(dir_parent)
                # print(dst_path)

Tokens in prompt:  4183
Tokens in completion:  405
Tokens in prompt:  4408
Tokens in completion:  381
Tokens in prompt:  4269
Tokens in completion:  401
Tokens in prompt:  4208
Tokens in completion:  435
Tokens in prompt:  4574
Tokens in completion:  492
Tokens in prompt:  4652
Tokens in completion:  27
Tokens in prompt:  4096
Tokens in completion:  312
Tokens in prompt:  4301
Tokens in completion:  448
Tokens in prompt:  4485
Tokens in completion:  500
Unterminated string starting at: line 1 column 1942 (char 1941)
API response could not be converted to dictionary. Returning the content of the completion message and the completion object as dict
Tokens in prompt:  4224
Tokens in completion:  470
Tokens in prompt:  3715
Tokens in completion:  401
Tokens in prompt:  4232
Tokens in completion:  351
Tokens in prompt:  4191
Tokens in completion:  395
Tokens in prompt:  3645
Tokens in completion:  426
Tokens in prompt:  3645
Tokens in completion:  43
Tokens in prompt:  4152
Tokens in comple

In [78]:
### Poland
SRC_DIR_po = './DPA-decisions/DPA-decisions/POLAND/' #full path to source directory
TRG_DIR_po = './poland_qr_results/'#full path to target directory
SUM_DIR_po = TRG_DIR_po
suffix_po = 'document.html'

### Poland (large document) ###

In [100]:
### large documents - Poland
summ_count = 0
num_big_files = 0
for root, dirs, files in os.walk(SUM_DIR_po):
    for f in files:
        if f.endswith(suffix_po):
            filepath = os.path.join(root, f)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR_po+grandpa
            dir_parent = TRG_DIR_po+grandpa+"/"+parent
            txt = read_html(filepath)
            token = ss.count_tokens(txt)
            if token > NORMAL_TOKEN_DOC and token < MAX_TOKEN_DOC:
                print(dir_parent)
                num_big_files += 1
                summ_count += 1
                print("----- big file with tokens:" + str(token) + " ------")
                #### large document pipeline
                segments = partition_document(txt)
                ### write summary
                print("--- directory: " + str(dir_parent)+ " ------")
                summary_chain = write_summary_long(segments,dir_parent,country="spain", save_result = True, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                # summary_aggr.append(summary_chain)
                continue
            elif token <= NORMAL_TOKEN_DOC:
                summ_count += 1
                ### write summary
                # summary, completion_dict = write_summary(txt,dir_parent,country="portugal", save_result = True,token_limit = MAX_TOKEN_DOC, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                # ### produce examples for JSON
                # ### write JSON
                # # if summ_count % 20 == 0:
                # print(" ----- SUMMARY FINISHED: "+  str(summ_count) + " -----")
            elif token >= MAX_TOKEN_DOC:
                summ_count += 1
                print("------------------------------------too huge file with tokens: " + str(token))
            
print("----- number of large document summaries: -----" + str(num_big_files))
print("----- total number of documents-----" + str(summ_count))

./poland_qr_results/DECYZJA/2020-12-03--DKN_5112_1_2020
----- big file with tokens:28175 ------
6
--- directory: ./poland_qr_results/DECYZJA/2020-12-03--DKN_5112_1_2020 ------
--- summary chain starts ------
Tokens in prompt:  5278
Tokens in completion:  470
--- chain at 1/6
Tokens in prompt:  5715
Tokens in completion:  678
--- chain at 2/6
Tokens in prompt:  5841
Tokens in completion:  651
--- chain at 3/6
Tokens in prompt:  5866
Tokens in completion:  771
--- chain at 4/6
Tokens in prompt:  6012
Tokens in completion:  720
--- chain at 5/6
Tokens in prompt:  5922
Tokens in completion:  679
Total number of summaries written: 6
./poland_qr_results/DECYZJA/2022-06-01--DKN_5131_2_2022
----- big file with tokens:17850 ------
4
--- directory: ./poland_qr_results/DECYZJA/2022-06-01--DKN_5131_2_2022 ------
--- summary chain starts ------
Tokens in prompt:  5082
Tokens in completion:  465
--- chain at 1/4
Tokens in prompt:  5685
Tokens in completion:  629
--- chain at 2/4
Tokens in prompt:  5

In [101]:
### large sized files --- Poland
large_summ_count = 0
c_json = 0
for root, dirs, files in os.walk(SUM_DIR_po):
    for f in files:
        if f.endswith(suffix_po):
            filepath = os.path.join(root, f)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR_po+grandpa
            dir_parent = TRG_DIR_po+grandpa+"/"+parent
            txt = read_html(filepath)
            token = ss.count_tokens(txt)
            if token > NORMAL_TOKEN_DOC and token < MAX_TOKEN_DOC:
                # print(save_dir)
                summary_path = dir_parent + "/summary_t1.0.txt"
                summa = read_txt(summary_path)
                large_summ_count += 1
                msg = construct_message_summary_to_json(summa, ppt.msg_sys_json, examples_summary, examples_json)
                res_dict, comp = summary_to_json(msg, dir_parent)
                if not isinstance(res_dict, str):
                    c_json += 1
                # print(res_dict)
                # res_list.append(res_dict)
                # comp_list.append(comp)
                if c_json % 20 == 0:
                    print(" ----- JSON FINISHED: "+  str(c_json) + " -----")
                    # print(summary)
                    # print(completion_dict)
                    # with open(dst_path,'w') as f: ### write the 
                    #     pass
                # print(dir_grand)
                # print(dir_parent)
                # print(dst_path)

Tokens in prompt:  4322
Tokens in completion:  499
Tokens in prompt:  4198
Tokens in completion:  425
Tokens in prompt:  4299
Tokens in completion:  423
Tokens in prompt:  4200
Tokens in completion:  387
Tokens in prompt:  4185
Tokens in completion:  495
Tokens in prompt:  4302
Tokens in completion:  500
Unterminated string starting at: line 1 column 2027 (char 2026)
API response could not be converted to dictionary. Returning the content of the completion message and the completion object as dict
Tokens in prompt:  4095
Tokens in completion:  314
Tokens in prompt:  4147
Tokens in completion:  388
Tokens in prompt:  4482
Tokens in completion:  498
Tokens in prompt:  4312
Tokens in completion:  419
Tokens in prompt:  4223
Tokens in completion:  436
Tokens in prompt:  4097
Tokens in completion:  384
Tokens in prompt:  4449
Tokens in completion:  448
Tokens in prompt:  4473
Tokens in completion:  463
Tokens in prompt:  4264
Tokens in completion:  500
Unterminated string starting at: line 